In [ ]:
!pip install pyvi unidecode gdown

In [ ]:
import pandas as pd
import numpy as np
from pyvi import ViTokenizer
import re
import unidecode
import gdown

gdown.download_folder('https://drive.google.com/drive/folders/1XpqF_ejSmQQJ4IsO38hJDZgMWLZelJyW?usp=sharing',quiet = False)

In [ ]:
corpus = pd.read_csv('/BKAI/corpus.csv')
public_test = pd.read_csv('/BKAI/public_test.csv')
train = pd.read_csv('/BKAI/train.csv')

In [ ]:
# Remove rows with NaN values and reset index of corpus dataframe
corpus = corpus.dropna().reset_index(drop=True)

# Remove rows with NaN values and reset index of public_test dataframe
public_test = public_test.dropna().reset_index(drop=True)

# Remove rows with NaN values and reset index of train dataframe
train = train.dropna().reset_index(drop=True)

In [ ]:
file1_path = '/AIO_RAGforJack/vietnamese-stopwords-dash.txt'  # Thay đổi đường dẫn tới file thứ nhất
file2_path = '/AIO_RAGforJack/vietnamese-stopwords.txt'  # Thay đổi đường dẫn tới file thứ hai
def read_stopwords(file_path):
    stopwords = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            stopword = line.strip()  # Tách theo khoảng trắng
            stopwords.append(stopword)  # Thêm stopword vào list
    return stopwords
stopwords1 = read_stopwords(file1_path)
stopwords2 = read_stopwords(file2_path)

vietnamese_stopwords = set(stopwords1 + stopwords2)

def remove_punctuation(text):
    no_punct_text = re.sub(r'[^\w\s]', '', text)  # This will keep Vietnamese letters and spaces
    return no_punct_text

In [ ]:
corpus['text'] = corpus['text'].apply(lambda x: x.replace('\n', ' '))
corpus['text'] = corpus['text'].apply(lambda x: x.replace('\t', ' '))
corpus['text'] = corpus['text'].apply(lambda x: x.replace('\r', ' '))
corpus['text'] = corpus['text'].apply(lambda x: x.replace('  ', ' '))
corpus['text'] = corpus['text'].apply(lambda x: x.lower())

public_test['question'] = public_test['question'].apply(lambda x: x.replace('\n', ' '))
public_test['question'] = public_test['question'].apply(lambda x: x.replace('\t', ' '))
public_test['question'] = public_test['question'].apply(lambda x: x.replace('\r', ' '))
public_test['question'] = public_test['question'].apply(lambda x: x.replace('  ', ' '))
public_test['question'] = public_test['question'].apply(lambda x: x.lower())

train['question'] = train['question'].apply(lambda x: x.replace('\n', ' '))
train['question'] = train['question'].apply(lambda x: x.replace('\t', ' '))
train['question'] = train['question'].apply(lambda x: x.replace('\r', ' '))
train['question'] = train['question'].apply(lambda x: x.replace('  ', ' '))
train['question'] = train['question'].apply(lambda x: x.lower())

train['context'] = train['context'].apply(lambda x: x.replace('\n', ' '))
train['context'] = train['context'].apply(lambda x: x.replace('\t', ' '))
train['context'] = train['context'].apply(lambda x: x.replace('\r', ' '))
train['context'] = train['context'].apply(lambda x: x.replace('  ', ' '))
train['context'] = train['context'].apply(lambda x: x.lower())

corpus['text'] = corpus['text'].apply(lambda x: ViTokenizer.tokenize(x))
corpus['text'] = corpus['text'].apply(lambda x: remove_punctuation(x))
corpus['text'] = corpus['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in vietnamese_stopwords]))
corpus['text'] = corpus['text'].apply(lambda x: unidecode.unidecode(x))

public_test['question'] = public_test['question'].apply(lambda x: ViTokenizer.tokenize(x))
public_test['question'] = public_test['question'].apply(lambda x: remove_punctuation(x))
public_test['question'] = public_test['question'].apply(lambda x: ' '.join([word for word in x.split() if word not in vietnamese_stopwords]))
public_test['question'] = public_test['question'].apply(lambda x: unidecode.unidecode(x))

train['question'] = train['question'].apply(lambda x: ViTokenizer.tokenize(x))
train['question'] = train['question'].apply(lambda x: remove_punctuation(x))
train['question'] = train['question'].apply(lambda x: ' '.join([word for word in x.split() if word not in vietnamese_stopwords]))
train['question'] = train['question'].apply(lambda x: unidecode.unidecode(x))

train['context'] = train['context'].apply(lambda x: ViTokenizer.tokenize(x))
train['context'] = train['context'].apply(lambda x: remove_punctuation(x))
train['context'] = train['context'].apply(lambda x: ' '.join([word for word in x.split() if word not in vietnamese_stopwords]))
train['context'] = train['context'].apply(lambda x: unidecode.unidecode(x))




In [ ]:
train.to_csv('train.csv', index=False)
public_test.to_csv('public_test.csv', index=False)
corpus.to_csv('corpus.csv', index=False)